# Метод первый - Явное нахождение название темы в словах
###### когда никак не получается найти датасет

### Будем парсить страничку, оставляя только слова и глаголы и в них искать слова из данных тематик
#### А выводить будем [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) значения в процентах

In [1]:
# данные тематики
t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")

# массив последних тем в иерархии (топики) и словарь, с пом. которого можно восстановить полную структуру
topics = [x[1:].split("/")[-1] for x in t]
restore = {topics[i]: t[i] for i in range(len(topics))}

In [2]:
topics

['Наука и образование',
 'наука',
 'математика',
 'физика',
 'химия',
 'информатика',
 'биоинформатика',
 'анализ данных',
 'литература',
 'образование',
 'школьное',
 'высшее',
 'дополнительное',
 'GoTo',
 'Политика',
 'Внутренняя',
 'Внешняя',
 'Экономика и бизнес',
 'Бизнес',
 'Стартапы',
 'E-Contenta',
 'Крупные компании',
 'Экономика',
 'Отдых и развлечения',
 'Кино',
 'Театр',
 'Компьютерные игры',
 'Фитнес',
 'Медицина',
 'Косметология']

In [3]:
import requests
import re
import nltk
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

import pymorphy2

### Firstly, we need to download the page we are looking for

In [4]:
web_url = "https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C_%D0%92%D0%B0%D0%BD%D0%B4%D0%B5%D1%80%D0%BC%D0%BE%D0%BD%D0%B4%D0%B0"
page = requests.get(web_url)
soup = BeautifulSoup(page.content, "lxml")

### Next step we clear text from punctuation, html tags and apply regular expression

In [5]:
h = HTML2Text()
h.ignore_links = True

In [6]:
raw_text = h.handle(soup.body.text)

In [7]:
only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()

### Now define a function

In [9]:
def get_nouns_list_from_url(web_url):
    # получим текст
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, "lxml")
    h = HTML2Text()
    h.ignore_links = True
    raw_text = h.handle(soup.body.text)
    only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
    
    # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форме
    morph = pymorphy2.MorphAnalyzer()
    words = only_russian_text.split()
    stoplist = stopwords.words('russian')
    data = []
    for word in words:
        if 2 < len(word) < 16 and word not in stoplist:
            p = morph.parse(word)[0]
            if p.tag.POS == 'NOUN' or "VERB":
                data.append(str(p.normal_form))
    
    return data

In [10]:
get_nouns_list_from_url(web_url)

['определитель',
 'вандермонд',
 'материал',
 'википедия',
 'свободный',
 'энциклопедия',
 'перейти',
 'навигация',
 'поиск',
 'определитель',
 'вандермонд',
 'называться',
 'определитель',
 'существовать',
 'равно',
 'степень',
 'иной',
 'слово',
 'делиться',
 'различный',
 'многочлен',
 'степень',
 'значит',
 'равный',
 'произведение',
 'точность',
 'константа',
 'убедиться',
 'раскрыть',
 'скобка',
 'константа',
 'равный',
 'единица',
 'дать',
 'формула',
 'показывать',
 'определитель',
 'вандермонд',
 'равный',
 'нуль',
 'существовать',
 'хотя',
 'один',
 'пара',
 'такой',
 'определитель',
 'вандермонд',
 'иметь',
 'многочисленный',
 'применение',
 'разный',
 'область',
 'математика',
 'например',
 'решение',
 'задача',
 'интерполяция',
 'многочлен',
 'задача',
 'нахождение',
 'многочлен',
 'степень',
 'график',
 'который',
 'проходить',
 'задать',
 'точка',
 'плоскость',
 'абсцисса',
 'определитель',
 'вандермонд',
 'возникать',
 'определитель',
 'система',
 'линейный',
 'уравнени

### It's time to select only nouns and turn them into normal form

In [12]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

### Now we need to get top tf-idf values and show corresponding words

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# инициализируем модельки
count_vect = CountVectorizer()
tf_idf = TfidfTransformer()

In [14]:
# применяем CountVectorizer к строке из слов, вытянутых с веб-страницы
X = count_vect.fit_transform([" ".join(get_nouns_list_from_url(web_url))])
X = tf_idf.fit_transform(X)

In [15]:
# отсортированный по невозрастанию массив пар индекс-tfidf значение
tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)

In [16]:
tfidf_repr[:10]

[(75, 0.32551538350846376),
 (3, 0.32551538350846376),
 (149, 0.28482596056990578),
 (92, 0.28482596056990578),
 (1, 0.20344711469278987),
 (52, 0.20344711469278987),
 (47, 0.20344711469278987),
 (127, 0.16275769175423188),
 (55, 0.16275769175423188),
 (133, 0.16275769175423188)]

In [17]:
# теперь сделаем из этого словарь
tfidf_dict = {id_ : tfidf for id_, tfidf in tfidf_repr}

In [18]:
# словарь для доставания слов из CountVectorizer словаря - то, что имеем
good_vocab = {}
for word, _id in count_vect.vocabulary_.items():
    good_vocab[str(_id)] = word

In [21]:
# хочется, чтобы сумма tf-idf значений в процентах была 100 :)
def softmax(X):
    a = np.array(X)
    multiplier = 100 / np.sum(a)
    return list(a * multiplier)

In [23]:
res = []
a = []
b = []
for k in topics:
    # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
    word = morph.parse(k)[0].normal_form
    _id = count_vect.vocabulary_.get(word)
    if _id:
        # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
        tfidf = tfidf_dict.get(_id)
        if tfidf:
            a.append(restore[k])
            b.append(tfidf)
b_soft = softmax(b)
# только сейчас добавляем в результирующий массив изначальное название топика и его процентное содержание
for i in range(len(a)):
    res.append("{} — {:.2f}%".format(a[i], b_soft[i]))
    
res = sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)

In [24]:
res[:10]

['    /Наука и образование/наука — 28.57%',
 '    /Наука и образование/наука/математика — 14.29%',
 '    /Наука и образование/наука/литература — 14.29%',
 '    /Наука и образование/образование/высшее — 14.29%',
 '    /Наука и образование/образование/дополнительное — 14.29%',
 '    /Политика — 14.29%']

### Now let's create all-in-one function (besause we can :) )

In [91]:
import requests
import re
import nltk
import pymorphy2
import numpy as np
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


def method1(web_url, topn=10):
    t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")
    topics = [x[1:].split("/")[-1] for x in t]
    restore = {topics[i]: t[i] for i in range(len(topics))}
    
    
    # получим текст
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, "lxml")
    h = HTML2Text()
    h.ignore_links = True
    raw_text = h.handle(soup.body.text)
    only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
    
    # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форме
    morph = pymorphy2.MorphAnalyzer()
    words = only_russian_text.split()
    stoplist = stopwords.words('russian')
    data = []
    for word in words:
        if 2 < len(word) < 16 and word not in stoplist:
            p = morph.parse(word)[0]
            if p.tag.POS == 'NOUN' or "VERB":
                data.append(str(p.normal_form))
                
    count_vect = CountVectorizer()
    tf_idf = TfidfTransformer()
    X = count_vect.fit_transform([" ".join(data)])
    X = tf_idf.fit_transform(X)
    
    tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)
    tfidf_dict = {id_ : tfidf for id_, tfidf in tfidf_repr}
    
    good_vocab = {str(_id) : word for word, _id in count_vect.vocabulary_.items()}
    
    res = []
    a = []
    b = []
    for k in topics:
        # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
        word = morph.parse(k)[0].normal_form
        _id = count_vect.vocabulary_.get(word)
        if _id:
            # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
            tfidf = tfidf_dict.get(_id)
            if tfidf:
                a.append(restore[k])
                b.append(tfidf)
                
    v = np.array(b)
    if np.sum(v) == 0:
        print("Вероятно, вы показываете мне английский текст, я так не умею")
        print("Ну или вы нашли сайт, который не подходит ни под одну категорию")
        return []
    multiplier = 100 / np.sum(v)
    b_soft = list(v * multiplier)

    # только сейчас добавляем в результирующий массив изначальное название топика и его процентное содержание
    for i in range(len(a)):
        res.append("{} — {:.2f}%".format(a[i], b_soft[i]))
        
    return sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)[:topn]

In [92]:
method1("http://goto.msk.ru/hackathon/")

['    /Наука и образование/образование — 31.58%',
 '    /Наука и образование/наука — 15.79%',
 '    /Здоровье и красота/Медицина — 15.79%',
 '    /Экономика и бизнес/Бизнес — 10.53%',
 '    /Наука и образование/образование/школьное — 5.26%',
 '    /Экономика и бизнес/Экономика — 5.26%',
 '    /Отдых и развлечения/Кино — 5.26%',
 '    /Отдых и развлечения/Театр — 5.26%',
 '    /Здоровье и красота/Фитнес — 5.26%']

In [93]:
method1("https://www.kinopoisk.ru/film/648440/")

['    /Отдых и развлечения/Кино — 75.00%',
 '    /Наука и образование/образование/высшее — 12.50%',
 '    /Наука и образование/образование — 6.25%',
 '    /Политика/Внешняя — 6.25%']

In [94]:
method1("https://e-contenta.com/ru/")

['    /Экономика и бизнес/Бизнес — 75.00%',
 '    /Наука и образование/образование/высшее — 25.00%']

In [95]:
method1("https://ru.wikipedia.org/wiki/C%2B%2B")

['    /Наука и образование/образование/высшее — 56.41%',
 '    /Наука и образование/образование/дополнительное — 10.26%',
 '    /Наука и образование/наука/математика — 7.69%',
 '    /Наука и образование/наука/литература — 7.69%',
 '    /Политика/Внешняя — 7.69%',
 '    /Наука и образование/наука/информатика — 5.13%',
 '    /Политика — 2.56%',
 '    /Политика/Внутренняя — 2.56%']

# Метод 2 - Использовать Word2Vec модель
#### Будем складывать векторы всех слов

## using [this](http://ling.go.mail.ru/misc/dialogue_2015.html) corpora
### (Russian Wikipedia together with Russian National Corpus)

In [25]:
from gensim.models.word2vec import Word2Vec

In [76]:
model = Word2Vec.load_word2vec_format('news.model.bin', binary=True)

In [77]:
a = []
b = []
for k in topics:
    # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
    word = morph.parse(k)[0].normal_form
    _id = count_vect.vocabulary_.get(word)
    if _id:
        # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
        tfidf = tfidf_dict.get(_id)
        if tfidf:
            a.append(k)
            b.append(tfidf)

In [78]:
k = 0
# нулевой вектор, он же и результирующий - сложение по правилам numpy
vector_summ = np.zeros(model.vector_size)
for i in range(len(a)):
    try:
        vector_summ += (model[a[i]] * b[i])
    except KeyError:
        k += 1

hundred_similar_words = model.most_similar(positive=[vector_summ], negative=[], topn=100)

In [79]:
hundred_similar_words

[('наука', 0.8318927884101868),
 ('математика', 0.6977936029434204),
 ('литература', 0.6834399700164795),
 ('физика', 0.6183444857597351),
 ('биология', 0.6048916578292847),
 ('естествознание', 0.5761309862136841),
 ('физико-математический', 0.5751566290855408),
 ('информатика', 0.5618166923522949),
 ('лингвистика', 0.5615713596343994),
 ('языкознание', 0.5526806116104126),
 ('культурология', 0.5335828065872192),
 ('филологический', 0.5316666960716248),
 ('обществознание', 0.531338632106781),
 ('химия', 0.5215914249420166),
 ('кибернетика', 0.5192195177078247),
 ('естественнонаучный', 0.517939031124115),
 ('обществоведение', 0.5163986682891846),
 ('литературоведение', 0.5140202641487122),
 ('член-корреспондент', 0.5103638172149658),
 ('астрономия', 0.5091902017593384),
 ('математик', 0.5085898041725159),
 ('алгебра', 0.501548707485199),
 ('педагогика', 0.49928444623947144),
 ('естественно-научный', 0.49542444944381714),
 ('доцент', 0.4946599304676056),
 ('география', 0.4937931299209595

In [80]:
restore[thousand_similar_words[0][0]], thousand_similar_words[0][1]

('    /Наука и образование/наука', 0.8318927884101868)

In [81]:
res = []
try:
    for i in range(len(restore)):
        similar_word = thousand_similar_words[i]
        res.append("{} — {:.2f}%".format(restore[similar_word[0]], similar_word[1] * 100))
except KeyError:
    pass
    
res = sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)

In [82]:
res

['    /Наука и образование/наука — 83.19%',
 '    /Наука и образование/наука/математика — 69.78%',
 '    /Наука и образование/наука/литература — 68.34%',
 '    /Наука и образование/наука/физика — 61.83%']

## Снова объединим все блоки и определим класс

In [4]:
import requests
import re
import nltk
import pymorphy2
import numpy as np
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from gensim.models.word2vec import Word2Vec


class Method2():
    def __init__(self):
        t = """/Наука и образование
        /Наука и образование/наука
        /Наука и образование/наука/математика
        /Наука и образование/наука/физика
        /Наука и образование/наука/химия
        /Наука и образование/наука/информатика
        /Наука и образование/наука/информатика/биоинформатика
        /Наука и образование/наука/информатика/анализ данных
        /Наука и образование/наука/литература
        /Наука и образование/образование
        /Наука и образование/образование/школьное
        /Наука и образование/образование/высшее
        /Наука и образование/образование/дополнительное
        /Наука и образование/образование/дополнительное/GoTo
        /Политика
        /Политика/Внутренняя
        /Политика/Внешняя
        /Экономика и бизнес
        /Экономика и бизнес/Бизнес
        /Экономика и бизнес/Бизнес/Стартапы
        /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
        /Экономика и бизнес/Бизнес/Крупные компании
        /Экономика и бизнес/Экономика
        /Отдых и развлечения
        /Отдых и развлечения/Кино
        /Отдых и развлечения/Театр
        /Отдых и развлечения/Компьютерные игры
        /Здоровье и красота/Фитнес
        /Здоровье и красота/Медицина
        /Здоровье и красота/Косметология""".split("\n")
        self.topics = [x[1:].split("/")[-1] for x in t]
        self.restore = {self.topics[i]: t[i] for i in range(len(self.topics))}

        self.morph = pymorphy2.MorphAnalyzer()

        self.model = Word2Vec.load_word2vec_format('ruscorpora.model.bin', binary=True)

    def classify(self, web_url, topn=10):

        # получим текст
        page = requests.get(web_url)
        soup = BeautifulSoup(page.content, "lxml")
        h = HTML2Text()
        h.ignore_links = True
        raw_text = h.handle(soup.body.text)
        only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
    
        # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форме
    
        words = only_russian_text.split()
        stoplist = stopwords.words('russian')
        data = []
        for word in words:
            if 2 < len(word) < 16 and word not in stoplist:
                p = self.morph.parse(word)[0]
                if p.tag.POS == 'NOUN' or "VERB":
                    data.append(str(p.normal_form))
    
        count_vect = CountVectorizer()
        tf_idf = TfidfTransformer()
        X = count_vect.fit_transform([" ".join(data)])
        X = tf_idf.fit_transform(X)
    
        tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)
        tfidf_dict = {id_: tfidf for id_, tfidf in tfidf_repr}
    
        a = []
        b = []
        for k in self.topics:
            # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
            word = self.morph.parse(k)[0].normal_form
            _id = count_vect.vocabulary_.get(word)
            if _id:
                # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
                tfidf = tfidf_dict.get(_id)
                if tfidf:
                    a.append(k)
                    b.append(tfidf)
    
        # нулевой вектор, он же и результирующий - сложение по правилам numpy
        vector_summ = np.zeros(self.model.vector_size)
        for i in range(len(a)):
            try:
                vector_summ += (self.model[a[i]] * b[i])
            except KeyError:
                pass
    
        hundred_similar_words = self.model.most_similar(positive=[vector_summ], negative=[], topn=1000)
    
        res = []
        try:
            for i in range(len(self.restore)):
                similar_word = hundred_similar_words[i]
                res.append("{} — {:.2f}%".format(self.restore[similar_word[0]], similar_word[1] * 100))
        except KeyError:
            pass
    
        return sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)[:topn]

In [5]:
m = Method2()
method2 = lambda web_url: m.classify(web_url)

In [86]:
method2("http://goto.msk.ru/hackathon/")

['        /Наука и образование/образование — 91.41%',
 '        /Наука и образование/наука — 58.53%']

In [87]:
method2("https://www.kinopoisk.ru/film/648440/")

[]

In [89]:
method2("https://e-contenta.com/ru/")

[]

In [7]:
method2("https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C_%D0%92%D0%B0%D0%BD%D0%B4%D0%B5%D1%80%D0%BC%D0%BE%D0%BD%D0%B4%D0%B0")

['        /Наука и образование/наука — 90.71%',
 '        /Наука и образование/наука/математика — 68.50%',
 '        /Наука и образование/наука/литература — 67.38%']

# Метод 3 - Использование линейной модели
###  Датасет с http://www.dmoz.org/

In [ ]:
from sklearn.cluster import KMeans

def kmeans(num_clusters, dataSet):
    # n_clusters: number of centroids
    # n_jobs: number of jobs running in parallel
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
   
    kmeansClustering = KMeans(n_clusters=num_clusters)
    # Compute cluster centers and predict cluster index for each sample
    centroidIndx = kmeansClustering.fit_predict(dataSet)

    return centroidIndx

def create_bag_of_centroids(reviewData,num_clusters,index_word_map):
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    """
        assign each word in the review to a centroid
        this returns a numpy array with the dimension as num_clusters
        each will be served as one feature for classification
        :param reviewData:
        :return:
    """
    featureVector = np.zeros(num_clusters, dtype=np.float)
    
    for word in reviewData:
        if word in index_word_map:
            index = index_word_map[word]
            featureVector[index] += 1
    
    return featureVector

#and here is the function for the classifier
def rfClassifer(n_estimators, trainingSet, label, testSet):
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    forest = RandomForestClassifier(n_estimators)
    forest = forest.fit(trainingSet, label)
    result = forest.predict(testSet)

    return result

In [ ]:
import pickle
import time
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import utilities.preProc as preProc
import utilities.classifierFuncs as cfun

modelName = "../../classifier/Word2VectforNLPTraining"
model = Word2Vec.load(open(modelName,"rb"))

# model.init_sims(replace=True)

wordVectors = model.syn0
# print(wordVectors[0])
num_clusters = int(wordVectors.shape[0] / 5)
# print("number of clusters: {}".format(num_clusters))
# input("Press enter to continue:")
print("Clustering...")
startTime = time.time()
clusterIndex = cfun.kmeans(num_clusters, wordVectors)
endTime = time.time()

print("Time taken for clustering: {} seconds".format(endTime - startTime))

clusterf = open("../../classifier/doc2vec/clusterIndex.pickle","wb") 

pickle.dump(clusterIndex,clusterf)

# create a word/index dictionary, mapping each vocabulary word to a cluster number
# zip(): make an iterator that aggregates elements from each of the iterables
index_word_map = dict(zip(model.index2word, clusterIndex))

train = pd.read_csv("../../data/labeledTrainData.tsv",
                header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../../data/testData.tsv",
               header=0, delimiter="\t", quoting=3)

trainingDataFV = np.zeros((train["review"].size, num_clusters), dtype=np.float)
testDataFV = np.zeros((test["review"].size, num_clusters), dtype=np.float)

# We dont really need to clean the data as the junk terms will be ignored anyway. This is due to the fact that we did not consider these while creating the model
# and hence they will not feature in the model's vocabulary. Still this step will expedite the classification and feature vector creation.
print("Processing training data...")
counter = 0
cleanedTrainingData = preProc.clean_data(train)
for review in cleanedTrainingData:
    trainingDataFV[counter] = cfun.create_bag_of_centroids(review,num_clusters,index_word_map)
    counter += 1

print("Processing test data...")
counter = 0
cleaned_test_data = preProc.clean_data(test)
for review in cleaned_test_data:
    testDataFV[counter] = cfun.create_bag_of_centroids(review,num_clusters,index_word_map)
    counter += 1

n_estimators = 100
result = cfun.rfClassifer(n_estimators, trainingDataFV, train["sentiment"],testDataFV)
output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
output.to_csv("Word2Vec_Clustering.csv", index=False, quoting=3)


### Now we got data for vectorizer, let's create SVM model

In [ ]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline

In [ ]:
count_vect = CountVectorizer(stop_words=stopwords.words('russian'))
hash_vect = HashingVectorizer()
tfidf_transformer = TfidfTransformer()
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)

In [ ]:
text_clf = Pipeline([('vect', count_vect),
                     ('tfidf', tfidf_transformer),
                     ('clf', sgd_clf)
])

In [ ]:
X = []
Y = []
for web_url, genre in ...:
    X.append(get_text_from_url(web_url))
    Y.append(genre)

In [ ]:
text_clf.fit(X, Y)